Note: This file has been used to evaluate predictions but is currently not in a suitable state for other users, it is more of a personal
playground than public evaluation tools.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import datetime
import h5py
from Network import loss_utils
from utils import evaluation_utils as e_utils
from skimage import morphology
from scipy import stats
from matplotlib import cm


plt.rcParams['figure.dpi'] = 1200

## Load files

In [ ]:
#data_dir = "../data"
#data_dir = "../THESIS_INVIVO_DATA/BRAIN/Healthy/HV01/h5"
data_dir = "../results/Bagging-12"

hr_filename = "P03_fft_SR.h5"
lr_filename = "patient3-postOp_LR.h5"
prediction_filename = "aorta03_SR.h5"

prediction_dirs = ["../results/combined", "../results/bag-12", "../results/halfsubbag-12"]

ground_truth_file = f"{data_dir}/{hr_filename}"
prediction_files = [f"{prediction_dir}/{prediction_filename}" for prediction_dir in prediction_dirs]
lr_file = f"{data_dir}/{lr_filename}"

# Parameters
mask_threshold = 0.25

In [ ]:
def save_to_h5(output_filepath, col_name, dataset, compression=None, delete=False):
    # convert float64 to float32 to save space
    if dataset.dtype == 'float64':
        dataset = np.array(dataset, dtype='float32')
    
    with h5py.File(output_filepath, 'a') as hf:
        if delete:
            del hf[col_name]
        hf.create_dataset(col_name, data=dataset, compression=compression) # gzip, compression_opts=4

In [ ]:
frame = 4
mask = None
with h5py.File("../THESIS_INVIVO_DATA/THORAX/P01/h5/P01_aorta_HR.h5", 'r') as hf:
        print(hf.keys())
        #print(np.array(hf['u_max']).shape)
        u = np.array(hf['u'])
        print(u.shape)
#         v = np.array(hf['v_max'])
#         w = np.array(hf['w_max'])
#         # if 'mask' in hf.keys():
#         #         mask = np.array(hf['mask'])
#         del hf['u_max']
#         del hf['v_max']
#         del hf['w_max']


# save_to_h5(f"{data_dir}/HV01_HR.h5", 'venc_u', u, compression='gzip')
# save_to_h5(f"{data_dir}/HV01_HR.h5", 'venc_v', v, compression='gzip')
# save_to_h5(f"{data_dir}/HV01_HR.h5", 'venc_w', w, compression='gzip')

# if mask.any():
#         save_to_h5(f"{data_dir}/P01_cardiac_frame_8.h5", 'mask', mask, compression='gzip')

In [ ]:
umax = []
vmax = []
wmax = []
frames = 3
for frame in range(frames):
    with h5py.File(ground_truth_file, 'r') as hf:
        u = np.array(hf['u'][frame])
        v = np.array(hf['v'][frame])
        w = np.array(hf['w'][frame])
    umax.append(np.amax(abs(u)))
    vmax.append(np.amax(abs(v)))
    wmax.append(np.amax(abs(w)))
umax = np.array(umax)
vmax = np.array(vmax)
wmax = np.array(wmax)
print(umax)
print(vmax)
print(wmax)
with h5py.File(ground_truth_file, 'a') as hf:
    hf.create_dataset('u_max', data=umax, compression='gzip')
    hf.create_dataset('v_max', data=vmax, compression='gzip')
    hf.create_dataset('w_max', data=wmax, compression='gzip')

## Load the mask

In [ ]:
temp_file = "../THESIS_INVIVO_DATA/THORAX/P03/h5/P03_aorta_HR.h5"
with h5py.File(temp_file, 'r') as hf:
    mask = tf.convert_to_tensor(hf['mask'])
    mask = mask[tf.newaxis] if len(mask.shape) == 3 else mask
    binary_mask = tf.cast((tf.cast(mask, dtype=tf.float32) >= mask_threshold), dtype=tf.float32)
    data_count = len(hf.get("u"))
print(mask.shape)

## Evaluation Tools

### Mean speed (Vx, Vy, Vz, |V|)

In [ ]:
mean_speed = np.zeros((data_count, 4))

peak_flow = -1
peak_flow_idx = -1

for idx in range(data_count):
    
    with h5py.File(ground_truth_file, mode = 'r' ) as hf:
        hr_u = tf.convert_to_tensor(hf['u'][idx])[tf.newaxis]
        hr_v = tf.convert_to_tensor(hf['v'][idx])[tf.newaxis]
        hr_w = tf.convert_to_tensor(hf['w'][idx])[tf.newaxis]    

    
    binary_mask_frame = binary_mask if binary_mask.shape[0] == 1 else binary_mask[idx][tf.newaxis]
    
    # Average speed per frame across all axis
    hr = tf.concat([hr_u, hr_v, hr_w], axis=0)
    
    squared = tf.map_fn(lambda x : tf.square(x), hr)
    speed = tf.math.sqrt(tf.reduce_sum(squared, axis=0))
    flow = tf.reduce_sum(speed, axis=[0,1,2]) / (tf.reduce_sum(binary_mask_frame, axis=[1,2,3]) + 1)*100

    # Average speed per frame for each axis independetly.
    flow_uvw = tf.reduce_sum(hr, axis=[1,2,3]) / (tf.reduce_sum(binary_mask_frame, axis=[1,2,3]) + 1)*100
    
    mean_speed[idx] = tf.concat([flow, flow_uvw], axis=0)
    if peak_flow < flow:
        peak_flow = flow
        peak_flow_idx = idx
print("Peak flow idx:", peak_flow_idx)
print("Plotting average speed...")
fig, ax = plt.subplots()
filename_start = prediction_files[0].find("-") + 1
colors = ['r', 'g', 'b', 'y']
labels = ['$\mathregular{|V|}$', '$\mathregular{V_x}$', '$\mathregular{V_y}$', '$\mathregular{V_z}$']
for i in range(4):
    ax.plot(tf.range(data_count), mean_speed[:, i], color=colors[i], label=labels[i])

ax.set_xlabel("Frames")
ax.set_ylabel("Avg. speed (cm/s)")
ax.legend()

plt.tight_layout()
#plt.savefig(f"../evaluation/{prediction_files[0][filename_start:-3]}_speed.png")
plt.show()


In [ ]:
mean_speed

### Relative mean error

In [ ]:
rel_err = np.zeros((len(prediction_files), data_count))
mean_speed = np.zeros((data_count, 4))

peak_flow = -1
peak_flow_idx = -1

for idx in range(data_count):
    # Load the ground truth U V W from H5 and crop if necessary
    for nr, prediction_file in enumerate(prediction_files):
        # Load the prediction U V W from H5
        with h5py.File(prediction_file, mode = 'r' ) as hf:
            pred_u = tf.convert_to_tensor(hf['u'][idx])[tf.newaxis]
            pred_v = tf.convert_to_tensor(hf['v'][idx])[tf.newaxis]
            pred_w = tf.convert_to_tensor(hf['w'][idx])[tf.newaxis]

        with h5py.File(ground_truth_file, mode = 'r' ) as hf:
            hr_u = tf.convert_to_tensor(hf['u'][idx])[tf.newaxis]
            hr_v = tf.convert_to_tensor(hf['v'][idx])[tf.newaxis]
            hr_w = tf.convert_to_tensor(hf['w'][idx])[tf.newaxis]

        # Relative error per frame
        
        rel_err[nr][idx] = (loss_utils.calculate_relative_error(pred_u, pred_v, pred_w, hr_u, hr_v, hr_w, binary_mask))
    
    
    # Average speed per frame across all axis
    hr = tf.concat([hr_u, hr_v, hr_w], axis=0)
    squared = tf.map_fn(lambda x : tf.square(x), hr)
    speed = tf.math.sqrt(tf.reduce_sum(squared, axis=0))
    flow = tf.reduce_sum(speed, axis=[0,1,2]) / (tf.reduce_sum(binary_mask, axis=[1,2,3]) + 1)*100

    # Average speed per frame for each axis independetly.
    flow_uvw = tf.reduce_sum(hr, axis=[1,2,3]) / (tf.reduce_sum(binary_mask, axis=[1,2,3]) + 1)*100
    
    mean_speed[idx] = tf.concat([flow, flow_uvw], axis=0)

# Plot
fig, ax = plt.subplots()
print(f"Plotting relative mean error...")

model_name_start = prediction_dirs[0].find("/" ,prediction_dirs[0].find("/")+1)+1

for idx, row in enumerate(rel_err):
    if data_count == 1:
        ax.scatter(0, row, label=prediction_dirs[idx][model_name_start:])
    else:
        ax.plot(np.arange(data_count), row, label=prediction_dirs[idx][model_name_start:])
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
ax.set_ybound(9, 35)
ax.set_xticks([0], 0)
ax.set_xlabel("Frame")
#plt.ylabel("Relative error (%)")
#plt.title(f"Relative speed error ({prediction_filename[:-6]})")
ax.legend()
fig.tight_layout()
plt.show()
#plt.savefig(f"../evaluation/{prediction_filename[:-3]}_new_RME.png", transparent=False, bbox_inches='tight')

### Qualitative slice comparison

In [ ]:
from scipy import ndimage

def get_slice_values(body, idx, axis='x'):
    if axis=='x':
        vals = body[idx, :, :]
    elif axis=='y':
        vals = body[:, idx, :]
    elif axis=='z':
        vals = body[:,:,idx]
    else:
        print("Error: x, y, z are available axes")
        return
    return vals

# Available vel_dirs are u, v, w.
def slice(file, frame, idx, vel_dir='u', axis='x'):

    with h5py.File(file, mode = 'r' ) as hf:
        body = np.asarray(hf[vel_dir][frame])
        sliced = get_slice_values(body, idx, axis)

    return sliced

# Available vel_dirs are u, v, w.
def slice_mul(file, frame, rng, vel_dir='u', slice_axis='x'):

    with h5py.File(file, mode = 'r' ) as hf:
        data = np.asarray(hf[vel_dir][frame])
    
    if slice_axis=='y':
        data = np.moveaxis(data, 1, 0)
    elif slice_axis=='z':
        data = np.moveaxis(data, 2, 0)
    
    return data[rng]


### RMSE and Relative Error in frame

In [ ]:
# Load files
data_dir = "../data"
prediction_dir = "../results/Bagging-2"

test_filenames = ["aorta03", "patient3-postOp", "cardiacM4Dx2"] # Assumed to be one file from each compartment
test_frames = [0,34,14]

# Parameters
mask_threshold = 0.25

In [ ]:
def calculate_error(hr, pred):
    e = (pred - hr) ** 2 
    
    # Return error and its standard deviation
    return e, np.std(e), np.var(e)

# Frame of interest
frame = 0

# Track RMSE and Variance across testing files / compartments
variance = {"u":0, "v": 0, "w": 0, "uvw": 0}
rmse = {"u":0, "v": 0, "w": 0, "uvw": 0}
total_rel = 0

model = prediction_dir[prediction_dir.rindex("/")+1:]
print(f"--- {model} prediction evaluation --- ")
# Calculate for all prediction files
for f, frame in zip(test_filenames, test_frames):
    ground_truth_file = f"{data_dir}/{f}_HR.h5"
    prediction_file = f"{prediction_dir}/{f}_SR.h5"

    print(f"File: {f}")
    # Load the prediction U V W from H5
    with h5py.File(prediction_file, mode = 'r' ) as hf:
        pred_u = tf.convert_to_tensor(hf['u'][frame])[tf.newaxis]
        pred_v = tf.convert_to_tensor(hf['v'][frame])[tf.newaxis]
        pred_w = tf.convert_to_tensor(hf['w'][frame])[tf.newaxis]

    with h5py.File(ground_truth_file, mode = 'r' ) as hf:
        hr_u = tf.convert_to_tensor(hf['u'][frame])[tf.newaxis]
        hr_v = tf.convert_to_tensor(hf['v'][frame])[tf.newaxis]
        hr_w = tf.convert_to_tensor(hf['w'][frame])[tf.newaxis]
        
        mask = tf.convert_to_tensor(hf['mask'])
        mask = mask[tf.newaxis] if len(mask.shape) == 3 else mask
        binary_mask = tf.cast((tf.cast(mask, dtype=tf.float32) >= mask_threshold), dtype=tf.float32)
        data_count = len(hf.get("u"))

    # Mean Tangent Absolute Percentage Error
    rel_err = (loss_utils.calculate_relative_error(pred_u, pred_v, pred_w, hr_u, hr_v, hr_w, binary_mask))
    total_rel += rel_err.numpy()[0]
    print(f"Relative Error: {rel_err.numpy()[0]:.2f}%")
    
    # Track error across velocity components for |V|
    e_uvw = 0
    for (component, pred, hr) in zip(['u', 'v', 'w'], [pred_u, pred_v, pred_w], [hr_u, hr_v, hr_w]):
        e, std, var = calculate_error(pred, hr)
        variance[component] += var
        rmse[component] += np.sqrt(np.mean(e))
        e_uvw += e
        print(f"{component} - RMSE + STD: {np.sqrt(np.mean(e)):.4f} + {std:.4f}")
    variance["uvw"] += np.var(e_uvw)
    rmse["uvw"] += np.sqrt(np.mean(e_uvw))
    print(f"Total RMSE + STD: {np.sqrt(np.mean(e_uvw)):.4f} + {np.std(e_uvw):.4f}")
    print("")
  
print("Average across compartments")
print(f"Relative error: {total_rel/len(test_filenames):.2f}%")
for (c, var), (_, rmse) in zip(variance.items(), rmse.items()):
    average_rmse = rmse / len(test_filenames)
    average_var = var / len(test_filenames)
    average_std = np.sqrt(average_var)
    print(f"{c} - RMSE + STD: {average_rmse:.4f} + {average_std:.4f}")


## Plot slices

In [ ]:
# Load files
data_dir ="../data"
eval_dirs = ["../results/Aorta", "../results/Cerebro", "../results/Cardiac", "../results/Combined", "../results/Bagging-2", "../results/Stacking-2"]#,"../results/Stacking-3-archblock"]# "../results/Bagging-12"]# "../results/Combined"]#,"../results/Stacking-2"]#, "../results/stacking-4-archupsample", "../results/stacking-3-block-4-upsample",]

test_filenames = [(0, 68, "aorta03"), (34, 96, "patient3-postOp"), (16, 20, "cardiacM4Dx2")] # Assumed to be one file from each compartment
invivo_filenames = [(1, 18, "HV01"), (5, 42, "P02_aorta"), (13, 26, "P03_cardiac")]
invivo2_filenames = [(5, 12, "HV03"),(5, 32, "P02_aorta"), (8, 32, "P02_cardiac")]
test_frames = [0,34,14]
test_file_idx = 2

invivo = False
if not invivo:
    frame, idx, test_file = test_filenames[test_file_idx] # in silico
else:
    data_dir = "../THESIS_INVIVO_DATA/BRAIN/Healthy/HV01/h5"
    #data_dir = "../THESIS_INVIVO_DATA/THORAX/P03/h5"
    frame, idx, test_file = invivo_filenames[test_file_idx] # in vivo

    
HR_file = f"{data_dir}/{test_file}_HR.h5"
LR_file = f"{data_dir}/{test_file}_LR.h5"
#LR_file = "../THESIS_INVIVO_DATA/THORAX/P01/h5/P01_fft_LR.h5"
delim = test_file.find('_')
test_file = test_file[:delim] if delim != -1 else test_file
test_file += "_HR_SR.h5" if invivo else "_SR.h5"
print(delim)
print(test_file)
HR_files = [f"{data_dir}/{domain[2]}_HR.h5" for domain in test_filenames]
test_files = [f"{domain[2]}_SR.h5" for domain in test_filenames]
eval_files = [f"{dir}/{test_file}" for dir in eval_dirs]
print(eval_files)
eval_files_for_all_HR = [[f"{dir}/{file}" for dir in eval_dirs] for file in test_files]


# Parameters
mask_threshold = 0.25

#frame = 0
#idx = 50
vel_dir = 'u'
slice_axis = 'x'
# Load mask
with h5py.File(HR_file, 'r') as hf:
    mask = tf.convert_to_tensor(hf['mask'])
    mask = mask[tf.newaxis] if len(mask.shape) == 3 else mask
    binary_mask = tf.cast((tf.cast(mask, dtype=tf.float32) >= mask_threshold), dtype=tf.float32)
    data_count = len(hf.get("u"))
print(mask.shape)


In [ ]:

# frame = 3
# idx = 20
#HR_file = "../THESIS_INVIVO_DATA/THORAX/P01/h5/P01_cardiac_HR.h5"

delim = "results/"
stop_delim = len(test_file) +1
start = eval_files[0].find(delim) + len(delim)
HR_slice = slice(HR_file, frame, idx, vel_dir, slice_axis)
HR_slice = HR_slice[50:132, 60:120]
print(HR_slice)
#LR_slice = slice(LR_file, frame, int(idx/2), vel_dir, slice_axis)
_min, _max = 99, -1
_min, _max = min(_min, np.amin(HR_slice)), max(_max, np.amax(HR_slice))
print(_min, _max)
#_min, _max = min(_min, np.amin(LR_slice)), max(_max, np.amax(LR_slice))
#print(_min, _max)
skip = 0
for file in eval_files:
    if skip:
        skip -= 1
        continue
    sliced = slice(file, frame, idx*2, vel_dir, slice_axis)
    _min, _max = min(_min, np.amin(sliced)), max(_max, np.amax(sliced))
    print(np.amin(sliced),np.amax(sliced))
#skip = 2
fig, ax = plt.subplots()
fig.tight_layout()
ax.set_axis_off()
print("Final:",_min, _max)
_min = -0.6
_max = 0.7
t = ax.imshow(HR_slice, interpolation='nearest', cmap='viridis', origin='lower', vmin=_min, vmax=_max)
#fig.colorbar(t, ax=ax, orientation="horizontal")
#fig.savefig(f"../evaluation/invivo/SRBeyond/HV01_1_18/NR_zoom1")
# fig, ax = plt.subplots()
# fig.tight_layout()
# ax.set_axis_off()
#test = ax.imshow(LR_slice, interpolation='nearest', cmap='viridis', origin='lower', vmin=_min, vmax=_max)
#fig.colorbar(test, ax=ax)
#fig.savefig(f"../evaluation/invivo/Recovery/P03_cardiac_13_26/LR")
index = 1
for file in eval_files:
    if skip:
        skip -= 1
        continue
    fig, ax = plt.subplots()
    fig.tight_layout()
    ax.set_axis_off()
    sliced = slice(file, frame, 2*idx, vel_dir, slice_axis)
    sliced = sliced[100:264,120:240]
    ax.imshow(sliced, interpolation='nearest', cmap='viridis', origin='lower', vmin=_min, vmax=_max)
    
    #fig.savefig(f"../evaluation/invivo/SRBeyond/HV01_1_18/{index}_zoom1")
    index += 1

# axis.imshow(LR_slice, interpolation='nearest', cmap='viridis', origin='lower', vmin=0, vmax=1)

# for ax, file in zip(axes.flatten(), eval_files):
#     sliced = slice(file, frame, idx, vel_dir, slice_axis)
#     diff_sliced = abs(HR_slice - sliced)
#     _min, _max = min(_min, np.amin(diff_sliced)), max(_max, np.amax(diff_sliced))
# test = []
# # for ax, file in zip(axes.flatten(), eval_files):
#     #ax.set_axis_off()
# file = eval_files[0]
# sliced = slice(file, frame, idx, vel_dir, slice_axis)
# diff_sliced = abs(HR_slice - sliced)
# test = axis.imshow(diff_sliced, interpolation='nearest', cmap='viridis', origin='lower', vmin=_min, vmax=_max)

#fig.colorbar(test, ax=axes[2])

plt.show()

### Plot absolute error

In [ ]:
delim = "results/"
stop_delim = len(test_file) +1
start = eval_files[0].find(delim) + len(delim)
HR_slice = slice(HR_file, frame, idx, vel_dir, slice_axis)
#LR_slice = slice(LR_file, frame, int(idx/2), vel_dir, slice_axis)
_min, _max = 99, -1
_min, _max = min(_min, np.amin(HR_slice)), max(_max, np.amax(HR_slice))
print(_min, _max)
#_min, _max = min(_min, np.amin(LR_slice)), max(_max, np.amax(LR_slice))
#print(_min, _max)
skip = 0
for file in eval_files:
    if skip:
        skip -= 1
        continue
    sliced = slice(file, frame, idx, vel_dir, slice_axis)
    sliced = abs(HR_slice-sliced)
    _min, _max = min(_min, np.amin(sliced)), max(_max, np.amax(sliced))
    print(np.amin(sliced),np.amax(sliced))
#skip = 2
fig, ax = plt.subplots()
fig.tight_layout()
ax.set_axis_off()
print("Final:",_min, _max)
_min = 0
_max = 1
t = ax.imshow(HR_slice, interpolation='nearest', cmap='viridis', origin='lower', vmin=_min, vmax=_max)
fig.colorbar(t, ax=ax, orientation="horizontal")
fig.savefig(f"../evaluation/insilico/baseVScombVSens/abs/Aorta03/colorbar")
# fig, ax = plt.subplots()
# fig.tight_layout()
# ax.set_axis_off()
#test = ax.imshow(LR_slice, interpolation='nearest', cmap='viridis', origin='lower', vmin=_min, vmax=_max)
#fig.colorbar(test, ax=ax)
#fig.savefig(f"../evaluation/invivo/Recovery/P03_cardiac_13_26/LR")
index = 1
for file in eval_files:
    print(file)
    if skip:
        skip -= 1
        continue
    fig, ax = plt.subplots()
    fig.tight_layout()
    ax.set_axis_off()
    sliced = slice(file, frame, idx, vel_dir, slice_axis)
    sliced = abs(HR_slice-sliced)
    ax.imshow(sliced, interpolation='nearest', cmap='viridis', origin='lower', vmin=_min, vmax=_max)
    
    f#ig.savefig(f"../evaluation/insilico/baseVScombVSens/abs/CardiacM4Dx2/cardiacM4Dx2_{index}")
    index += 1

# axis.imshow(LR_slice, interpolation='nearest', cmap='viridis', origin='lower', vmin=0, vmax=1)

# for ax, file in zip(axes.flatten(), eval_files):
#     sliced = slice(file, frame, idx, vel_dir, slice_axis)
#     diff_sliced = abs(HR_slice - sliced)
#     _min, _max = min(_min, np.amin(diff_sliced)), max(_max, np.amax(diff_sliced))
# test = []
# # for ax, file in zip(axes.flatten(), eval_files):
#     #ax.set_axis_off()
# file = eval_files[0]
# sliced = slice(file, frame, idx, vel_dir, slice_axis)
# diff_sliced = abs(HR_slice - sliced)
# test = axis.imshow(diff_sliced, interpolation='nearest', cmap='viridis', origin='lower', vmin=_min, vmax=_max)

#fig.colorbar(test, ax=axes[2])

plt.show()

In [ ]:
core_mask = morphology.binary_erosion(binary_mask)
boundary_mask = binary_mask - core_mask

Absolute error GIF, across slice axis

In [ ]:
fig, axes = plt.subplots(1,3)
print(axes)
delim = "results/"
stop_delim = len(test_file) +1
start = eval_files[1].find(delim) + len(delim)
LR_slice = slice(LR_file, frame, int(idx/2), vel_dir, slice_axis)
_min, _max = np.amin(LR_slice), np.amax(LR_slice)
print(_min, _max)

HR_slice = slice(HR_file, frame, idx, vel_dir, slice_axis)
_min, _max = min(_min, np.amin(HR_slice)), max(_max, np.amax(HR_slice))
print(_min, _max)
skip = 0

# Check max and min to set boundaries

for ax, file in zip(axes.flatten(), eval_files):
    if skip:
        skip -= 1
        continue
    sliced = slice(file, frame, idx, vel_dir, slice_axis)
    _min, _max = min(_min, np.amin(sliced)), max(_max, np.amax(sliced))
    print(file,_min, _max)



skip = 0
#axes[0].set_title("HR")
axes[0].set_axis_off()
axes[0].imshow(LR_slice, interpolation='nearest', cmap='viridis', origin='lower', vmin=_min, vmax=_max)
axes[1].set_axis_off()
axes[1].imshow(HR_slice, interpolation='nearest', cmap='viridis', origin='lower', vmin=_min, vmax=_max)
test = []
for ax, file in zip(axes.flatten(), eval_files):
    if skip:
        skip -= 1
        continue
    #ax.set_title(file[start:-stop_delim])
    ax.set_axis_off()
    sliced = slice(file, frame, idx, vel_dir, slice_axis)
    test = ax.imshow(sliced, interpolation='nearest', cmap='viridis', origin='lower')#, vmin=_min, vmax=_max)

# fig, ax = plt.subplots()
# ax.set_axis_off()
# ax.set_title("LR")
# test = ax.imshow(LR_slice, interpolation='nearest', cmap='viridis', origin='lower', vmin=_min, vmax=_max)
fig.tight_layout()
#fig.colorbar(test, ax=axes[4])

## Plot regression plots

In [ ]:
# Functions

def crop(hr, pred):
    # We assume that if there is a mismatch it's because SR is smaller than HR.
    crop = np.asarray(hr.shape) - np.asarray(pred.shape)
    hr = hr[crop[0]//2:-crop[0]//2,:,:] if crop[0] else hr
    hr = hr[:, crop[1]//2:-crop[1]//2,:] if crop[1] else hr
    hr = hr[:, :, crop[2]//2:-crop[2]//2] if crop[2] else hr
    return hr

def _reg_stats(subplot, hr_vals, sr_vals, color, x, y):
    x_text = x
    reg = stats.linregress(hr_vals, sr_vals)
    x = np.array([-10, 10]) # Start, End point for the regression slope lines
    if reg.intercept < 0.0:
        reg_stats = f'$y = {reg.slope:.3f}x {reg.intercept:.3f}$\n$r^2 = {reg.rvalue**2:.3f}$'
    else:
        reg_stats = f'$y = {reg.slope:.3f}x + {reg.intercept:.3f}$\n$r^2 = {reg.rvalue**2:.3f}$'
    subplot.plot(x, reg.intercept + reg.slope*x, color=color, linestyle='--', alpha=0.3)
    subplot.text(x_text, y, reg_stats, horizontalalignment='center', verticalalignment='center', fontsize=7, color=color)

def _setup_lin_reg_plot(fig_nr, xlim, ylim, title):
    plt.figure(fig_nr)
    fig, subplots = plt.subplots(1,3,sharey=True, figsize=plt.figaspect(1/3))
    dimensions = ["x", "y", "z"]
    fig.suptitle(title, fontsize=16)
    for i, subplot in enumerate(subplots):
        subplot.set_xlim(-xlim,xlim); subplot.set_ylim(-ylim,ylim); subplot.set_xticks([-xlim, xlim]); subplot.set_yticks([-ylim, ylim])
        subplot.set_title(f"$V_{{{dimensions[i]}}}$"); subplot.set_xlabel("$V_{HR}$ [m/s]"); subplot.set_ylabel("$V_{SR}$ [m/s]")
    
    return fig, subplots


def _plot_data(subplot, hr_vals, sr_vals, color, size, label):
    subplot.scatter(hr_vals, sr_vals, s=size, c=[color], label=label)
    subplot.legend(loc='lower right', fontsize=7)
    
    


def _sample_hrsr(ground_truth_file, prediction_file, mask, peak_flow_idx, samples):
    # Use mask to find interesting samples
    sample_pot = np.where(mask == 1)
    rng = np.random.default_rng()
    # Sample <ratio> samples
    print("pot:", len(sample_pot[0]))
    print("samples:", samples)
    sample_idx = rng.choice(len(sample_pot[0]), replace=False, size=samples)
    
    # Get indexes
    x_idx = sample_pot[0][sample_idx]
    y_idx = sample_pot[1][sample_idx]
    z_idx = sample_pot[2][sample_idx]

    with h5py.File(prediction_file, mode = 'r' ) as hf:
        sr_u = np.asarray(hf['u'][peak_flow_idx])
        sr_u_all = sr_u[sample_pot[0], sample_pot[1], sample_pot[2]]
        sr_u_samples = sr_u[x_idx, y_idx, z_idx]
        sr_v = np.asarray(hf['v'][peak_flow_idx])
        sr_v_all = sr_v[sample_pot[0], sample_pot[1], sample_pot[2]]
        sr_v_samples = sr_v[x_idx, y_idx, z_idx]
        sr_w = np.asarray(hf['w'][peak_flow_idx])
        sr_w_all = sr_w[sample_pot[0], sample_pot[1], sample_pot[2]]
        sr_w_samples = sr_w[x_idx, y_idx, z_idx]
        
    with h5py.File(ground_truth_file, mode = 'r' ) as hf:
        # Get velocity values in all directions
        hr_u = crop(np.asarray(hf['u'][peak_flow_idx]), sr_u)
        hr_u_all = hr_u[sample_pot[0], sample_pot[1], sample_pot[2]]
        hr_u_samples = hr_u[x_idx, y_idx, z_idx]
        hr_v = crop(np.asarray(hf['v'][peak_flow_idx]), sr_v)
        hr_v_all = hr_v[sample_pot[0], sample_pot[1], sample_pot[2]]
        hr_v_samples = hr_v[x_idx, y_idx, z_idx]
        hr_w = crop(np.asarray(hf['w'][peak_flow_idx]), sr_w)
        hr_w_all = hr_w[sample_pot[0], sample_pot[1], sample_pot[2]]
        hr_w_samples = hr_w[x_idx, y_idx, z_idx]
        
        
    return [hr_u_samples, hr_v_samples, hr_w_samples], [sr_u_samples, sr_v_samples, sr_w_samples], [hr_u_all, hr_v_all, hr_w_all], [sr_u_all, sr_v_all, sr_w_all]
    

def draw_multi_reg_line(ground_truth_file, prediction_dirs, peak_flow_idx, binary_mask, prediction_filename):
    """ Plot a linear regression between HR and predicted SR in peak flow frame """
    #
    # Parameters
    #

    # Hard coded
    boundary_voxels = 3000
    core_voxels = 3032
    voxels = 1000

    core_fig_nr = 1
    boundary_fig_nr = 2
    xlim = 1.0
    ylim = 1.0

    colors = ["red", "green", "blue", "orange", "purple", "black"]
    x = -2*xlim/3
    y = [4*ylim/5, 3*ylim/5, 2*ylim/5, ylim/5, 0, -ylim/5]
    
    # core_mask = morphology.binary_erosion(binary_mask)
    # boundary_mask = binary_mask - core_mask

    model_name_start = prediction_dirs[0].find("/" ,prediction_dirs[0].find("/")+1)+1
    
    # core_fig, core_subplots = _setup_lin_reg_plot(core_fig_nr, xlim, ylim, f"Core Voxels - {prediction_filename[:-6]}")
    # boundary_fig, boundary_subplots = _setup_lin_reg_plot(boundary_fig_nr, xlim, ylim, f"Boundary Voxels - {prediction_filename[:-6]}")
    fig, subplots = _setup_lin_reg_plot(1, xlim, ylim, f"Voxels - {prediction_filename[:-6]}")
    print(len(subplots))
    
    print(f"Plotting regression lines...")
    for i, prediction_dir in enumerate(prediction_dirs):
        prediction_file = f"{prediction_dir}/{prediction_filename}"
        print(prediction_file)
        #boundary_hr, boundary_sr = _sample_hrsr(ground_truth_file, prediction_file, boundary_mask, peak_flow_idx, boundary_voxels)
        hr_samples, sr_samples, hr, sr = _sample_hrsr(ground_truth_file, prediction_file, binary_mask, peak_flow_idx, voxels)
        _plot_data(subplots[0], hr_samples[0], sr_samples[0], colors[i], 0.5, label=prediction_dir[model_name_start:])
        _reg_stats(subplots[0], hr[0], sr[0], colors[i], x, y[i])
        _plot_data(subplots[1], hr_samples[1], sr_samples[1], colors[i], 0.5, label=prediction_dir[model_name_start:])
        _reg_stats(subplots[1], hr[1], sr[1], colors[i], x, y[i])
        _plot_data(subplots[2], hr_samples[2], sr_samples[2], colors[i], 0.5, label=prediction_dir[model_name_start:])
        _reg_stats(subplots[2], hr[2], sr[2], colors[i], x, y[i])
        
        # _plot_data(boundary_subplots[0], boundary_hr[0], boundary_sr[0], x, y[i], colors[i], 0.5, label=prediction_dir[model_name_start:])
        # _plot_data(boundary_subplots[1], boundary_hr[1], boundary_sr[1], x, y[i], colors[i], 0.5, label=prediction_dir[model_name_start:])
        # _plot_data(boundary_subplots[2], boundary_hr[2], boundary_sr[2], x, y[i], colors[i], 0.5, label=prediction_dir[model_name_start:])
        timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
    #fig.savefig(f"../evaluation/{prediction_filename[:-3]}_aorta_{timestamp}_reg.png") 
    #boundary_fig.savefig(f"../evaluation/{prediction_filename[:-3]}_boundary_{timestamp}_reg.png")

def draw_reg_line(ground_truth_file, prediction_dirs, peak_flow_idx, binary_mask, prediction_filename):
    """ Plot a linear regression between HR and predicted SR in peak flow frame """
    #
    # Parameters
    #

    # Hard coded
    voxels = 2500

    core_fig_nr = 1
    boundary_fig_nr = 2
    xlim = 1.0
    ylim = 1.0

    colors = [u'#1f77b4', u'#ff7f0e', u'#2ca02c', u'#d62728', u'#9467bd', u'#8c564b']
    x = -2*xlim/3
    y = [4*ylim/5, 3*ylim/5, 2*ylim/5, ylim/5, 0, -ylim/5]

    model_name_start = prediction_dirs[0].find("/" ,prediction_dirs[0].find("/")+1)+1
    
    
    fig, ax = plt.subplots()
    ax.set_xlim(-xlim,xlim); ax.set_ylim(-ylim,ylim); ax.set_xticks([-xlim, xlim]); ax.set_yticks([-ylim, ylim])
    ax.set_xlabel("$V_{HR}$ [m/s]"); #subplot.set_ylabel("$V_{SR}$ [m/s]"); ax.set_title(f"$V_{{{dimensions[i]}}}$"); 
    
    
    #boundary_fig, boundary_subplots = _setup_lin_reg_plot(boundary_fig_nr, xlim, ylim, f"Boundary Voxels - {prediction_filename[:-6]}")

    
    
    print(f"Plotting regression lines...")
    for i, prediction_dir in enumerate(prediction_dirs):
        prediction_file = f"{prediction_dir}/{prediction_filename}"
        hr, sr = _sample_hrsr(ground_truth_file, prediction_file, binary_mask, peak_flow_idx, voxels)
        #core_hr, core_sr = _sample_hrsr(ground_truth_file, prediction_file, core_mask, peak_flow_idx, core_voxels)
        _plot_data(ax, hr[0], sr[0], x, y[i], colors[i], 0.5, label=prediction_dir[model_name_start:])
    #core_fig.savefig(f"../evaluation/{prediction_filename[:-3]}_core_{timestamp}_reg.png") 
    #boundary_fig.savefig(f"../evaluation/{prediction_filename[:-3]}_boundary_{timestamp}_reg.png") 
     
    



In [ ]:
with h5py.File(eval_files[0], mode = 'r') as hf:
    u = tf.convert_to_tensor(hf['u'][0])

with h5py.File(HR_file, 'r') as hf:
    mask = tf.convert_to_tensor(hf['mask'])
    if len(mask.shape) == 3: 
        mask = crop(mask, u)[tf.newaxis]
    else:
        mask = crop(mask[frame], u)[tf.newaxis]
    # Casting excessively because eager tensors won't dynamically cast. 
    binary_mask = tf.cast((tf.cast(mask, dtype=tf.float32) >= mask_threshold), dtype=tf.float32)
    data_count = len(hf.get("u"))
        
print(HR_file, test_file, frame, eval_dirs)
draw_multi_reg_line(HR_file, eval_dirs, frame, tf.squeeze(binary_mask, axis=[0]), test_file)
plt.show()
#draw_reg_line(HR_file, eval_dirs, frame, tf.squeeze(binary_mask, axis=[0]), test_file)

In [ ]:
# Functions

def crop(hr, pred):
    # We assume that if there is a mismatch it's because SR is smaller than HR.
    crop = np.asarray(hr.shape) - np.asarray(pred.shape)
    hr = hr[crop[0]//2:-crop[0]//2,:,:] if crop[0] else hr
    hr = hr[:, crop[1]//2:-crop[1]//2,:] if crop[1] else hr
    hr = hr[:, :, crop[2]//2:-crop[2]//2] if crop[2] else hr
    return hr

def _reg_stats(subplot, hr_vals, sr_vals, color, x, y):
    x_text = x
    reg = stats.linregress(hr_vals, sr_vals)
    x = np.array([-10, 10]) # Start, End point for the regression slope lines
    if reg.intercept < 0.0:
        reg_stats = f'$y = {reg.slope:.3f}x {reg.intercept:.3f}$\n$r^2 = {reg.rvalue**2:.3f}$'
    else:
        reg_stats = f'$y = {reg.slope:.3f}x + {reg.intercept:.3f}$\n$r^2 = {reg.rvalue**2:.3f}$'
    subplot.plot(x, reg.intercept + reg.slope*x, color=color, linestyle='--', alpha=0.3)
    subplot.text(x_text, y, reg_stats, horizontalalignment='center', verticalalignment='center', fontsize=9, color=color)

def _setup_lin_reg_plot(fig_nr, xlim, ylim):
    plt.figure(fig_nr)
    fig, subplots = plt.subplots(1,3,sharey=True, figsize=plt.figaspect(1/3))
    dimensions = ["x", "y", "z"]
    for i, subplot in enumerate(subplots):
        subplot.set_xlim(-xlim,xlim)
        subplot.set_ylim(-ylim,ylim)
        subplot.set_xticks([])
        
        subplot.set_yticks([-ylim, ylim])
        #subplot.set_title(f"$V_{{{dimensions[i]}}}$"); 
        subplot.set_xlabel("$V_{HR}$ [m/s]"); subplot.set_xticks([-xlim,xlim])
        if i == 0:
            subplot.set_ylabel("$V_{SR}$ [m/s]")
    
    return fig, subplots


def _plot_data(subplot, hr_vals, sr_vals, color, marker, size, label):
    subplot.scatter(hr_vals, sr_vals, s=size, c=[color], label=label, marker=marker)
    #subplot.legend(loc='lower right', fontsize=7)
    
    


def _sample_hrsr(ground_truth_file, prediction_file, mask, peak_flow_idx, samples):
    # Use mask to find interesting samples
    sample_pot = np.where(mask == 1)
    rng = np.random.default_rng()
    # Sample <ratio> samples
    print("pot:", len(sample_pot[0]))
    print("samples:", samples)
    sample_idx = rng.choice(len(sample_pot[0]), replace=False, size=samples)
    
    # Get indexes
    x_idx = np.array_split(sample_pot[0][sample_idx], 3)
    y_idx = np.array_split(sample_pot[1][sample_idx], 3)
    z_idx = np.array_split(sample_pot[2][sample_idx], 3)

    with h5py.File(prediction_file, mode = 'r' ) as hf:
        sr_u = np.asarray(hf['u'][peak_flow_idx])
        sr_u_all = sr_u[sample_pot[0], sample_pot[1], sample_pot[2]]
        sr_u_samples = sr_u[x_idx[0], y_idx[0], z_idx[0]]

        sr_v = np.asarray(hf['v'][peak_flow_idx])
        sr_v_all = sr_v[sample_pot[0], sample_pot[1], sample_pot[2]]
        sr_v_samples = sr_v[x_idx[1], y_idx[1], z_idx[1]]

        sr_w = np.asarray(hf['w'][peak_flow_idx])
        sr_w_all = sr_w[sample_pot[0], sample_pot[1], sample_pot[2]]
        sr_w_samples = sr_w[x_idx[2], y_idx[2], z_idx[2]]
        
    with h5py.File(ground_truth_file, mode = 'r' ) as hf:
        # Get velocity values in all directions
        hr_u = crop(np.asarray(hf['u'][peak_flow_idx]), sr_u)
        hr_u_all = hr_u[sample_pot[0], sample_pot[1], sample_pot[2]]
        hr_u_samples = hr_u[x_idx[0], y_idx[0], z_idx[0]]

        hr_v = crop(np.asarray(hf['v'][peak_flow_idx]), sr_v)
        hr_v_all = hr_v[sample_pot[0], sample_pot[1], sample_pot[2]]
        hr_v_samples = hr_v[x_idx[1], y_idx[1], z_idx[1]]
        
        hr_w = crop(np.asarray(hf['w'][peak_flow_idx]), sr_w)
        hr_w_all = hr_w[sample_pot[0], sample_pot[1], sample_pot[2]]
        hr_w_samples = hr_w[x_idx[2], y_idx[2], z_idx[2]]
        
        
    return np.concatenate((hr_u_samples, hr_v_samples, hr_w_samples), axis=None), np.concatenate((sr_u_samples, sr_v_samples, sr_w_samples), axis=None), np.concatenate((hr_u_all, hr_v_all, hr_w_all), axis=None), np.concatenate((sr_u_all, sr_v_all, sr_w_all), axis=None)
    

def draw_multi_reg_line(ground_truth_file, prediction_dirs, peak_flow_idx, binary_mask, prediction_filename, subplot):
    """ Plot a linear regression between HR and predicted SR in peak flow frame """
    #
    # Parameters
    #

    # Hard coded
    voxels = 5000
    xlim = 1.0
    ylim = 1.0
    viridis = cm.get_cmap('viridis', 12)
    inferno = cm.get_cmap('inferno', 12)
    plasma = cm.get_cmap('plasma', 12)

    # colors = [viridis(0.9), viridis(0.5), inferno(0.5), viridis(0.), "orange", "purple", "black"]
    colors = [inferno(0.),inferno(0.5), viridis(0.3), viridis(0.6), "orange", "purple", "black"]
    markers = ['.', '1', '*', '+']
    x = -2*xlim/4
    y = [4*ylim/5, 3*ylim/5, 2*ylim/5, ylim/5, 0, -ylim/5]
    

    model_name_start = prediction_dirs[0].find("/" ,prediction_dirs[0].find("/")+1)+1
    
    print(f"Plotting regression lines...")
    for i, prediction_dir in enumerate(prediction_dirs):
        prediction_file = f"{prediction_dir}/{prediction_filename}"
        print(prediction_file)
        hr_samples, sr_samples, hr, sr = _sample_hrsr(ground_truth_file, prediction_file, binary_mask, peak_flow_idx, voxels)
        _plot_data(subplot, hr_samples, sr_samples, colors[i], markers[i], 0.5, label=f"4DFlowNet-{prediction_dir[model_name_start:]}")
        _reg_stats(subplot, hr, sr, colors[i], x, y[i])
        
        timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M")
    #fig.savefig(f"../evaluation/{prediction_filename[:-3]}_aorta_{timestamp}_reg.png") 
    #boundary_fig.savefig(f"../evaluation/{prediction_filename[:-3]}_boundary_{timestamp}_reg.png")





In [ ]:
xlim = 1.0
ylim = 1.0
fig, subplots = _setup_lin_reg_plot(1, xlim, ylim)
i = 0
for HR_ in HR_files:
    with h5py.File(eval_files_for_all_HR[i][0], mode = 'r') as hf:
        u = tf.convert_to_tensor(hf['u'][0])

    with h5py.File(HR_, 'r') as hf:
        mask = tf.convert_to_tensor(hf['mask'])
        # Casting excessively because eager tensors won't dynamically cast. 
        binary_mask = tf.cast((tf.cast(mask, dtype=tf.float32) >= mask_threshold), dtype=tf.float32)
        data_count = len(hf.get("u"))
            
    print(HR_file, test_file, frame, eval_dirs)
    draw_multi_reg_line(HR_, eval_dirs, test_frames[i], binary_mask, test_files[i], subplots[i])
    i += 1
fig.tight_layout()
fig.savefig(f"../evaluation/insilico/NumberOfLearners/Regressions/bagging_reg.png")
plt.show()

In [ ]:
def get_coefficients(ground_truth_file, prediction_file, mask, peak_flow_idx):
    
    # Use mask to find interesting samples
    sample_pot = np.where(mask == 1)

    with h5py.File(prediction_file, mode = 'r' ) as hf:
        sr_u = np.asarray(hf['u'][peak_flow_idx])
        sr_u_all = sr_u[sample_pot[0], sample_pot[1], sample_pot[2]]

        sr_v = np.asarray(hf['v'][peak_flow_idx])
        sr_v_all = sr_v[sample_pot[0], sample_pot[1], sample_pot[2]]

        sr_w = np.asarray(hf['w'][peak_flow_idx])
        sr_w_all = sr_w[sample_pot[0], sample_pot[1], sample_pot[2]]
        
    with h5py.File(ground_truth_file, mode = 'r' ) as hf:
        # Get velocity values in all directions
        hr_u = crop(np.asarray(hf['u'][peak_flow_idx]), sr_u)
        hr_u_all = hr_u[sample_pot[0], sample_pot[1], sample_pot[2]]
        hr_v = crop(np.asarray(hf['v'][peak_flow_idx]), sr_v)
        hr_v_all = hr_v[sample_pot[0], sample_pot[1], sample_pot[2]]
        hr_w = crop(np.asarray(hf['w'][peak_flow_idx]), sr_w)
        hr_w_all = hr_w[sample_pot[0], sample_pot[1], sample_pot[2]]
        
        
    return [hr_u_all, hr_v_all, hr_w_all], [sr_u_all, sr_v_all, sr_w_all]
    
def get_reg_stats(hr_vals, sr_vals):
    slope = []
    rsquared = []
    for hr, sr in zip(hr_vals, sr_vals):
        reg = stats.linregress(hr, sr)
        slope.append(np.round(reg.slope,3))
        rsquared.append(np.round(reg.rvalue**2, 3))
    return tuple(slope), tuple(rsquared)

xlim = 1.0
ylim = 1.0
i = 0
for HR_ in HR_files:
    with h5py.File(eval_files_for_all_HR[i][0], mode = 'r') as hf:
        u = tf.convert_to_tensor(hf['u'][0])

    with h5py.File(HR_, 'r') as hf:
        mask = tf.convert_to_tensor(hf['mask'])
        # Casting excessively because eager tensors won't dynamically cast. 
        binary_mask = tf.cast((tf.cast(mask, dtype=tf.float32) >= mask_threshold), dtype=tf.float32)
        data_count = len(hf.get("u"))
            
    print("Ground Truth file:", HR_)
    for prediction_dir in eval_dirs:
        prediction_file = f"{prediction_dir}/{test_files[i]}"
        print("Prediction file:",  prediction_file)
        hr, sr = get_coefficients(HR_,prediction_file,binary_mask,test_frames[i])
        slope, rsquared = get_reg_stats(hr, sr)
        print("Slope:", slope)
        print("R^2:", rsquared)

    
    i += 1
plt.show()

# K-values and R^2-values

In [ ]:
domain = "cardiac"
#model = "Bagging-12" # Best Bagging
model = "Stacking-3-archblock" # Best stacking

if domain == "aorta":
    predictions = [f"../results/{model}/P01_fft_SR.h5",
               f"../results/{model}/P02_fft_SR.h5",
               f"../results/{model}/P03_fft_SR.h5",
               f"../results/{model}/P04_fft_SR.h5",
               f"../results/{model}/P05_fft_SR.h5",]
    HR = ["../THESIS_INVIVO_DATA/THORAX/P01/h5/P01_aorta_HR.h5",
               "../THESIS_INVIVO_DATA/THORAX/P02/h5/P02_aorta_HR.h5",
               "../THESIS_INVIVO_DATA/THORAX/P03/h5/P03_aorta_HR.h5",
               "../THESIS_INVIVO_DATA/THORAX/P04/h5/P04_aorta_HR.h5",
               "../THESIS_INVIVO_DATA/THORAX/P05/h5/P05_aorta_HR.h5",]
    names = ["P01", "P02", "P03", "P04", "P05"]
    peak_frames = [4, 5, 1, 6, 5]
    frames = 25
elif domain == "cardiac":
    predictions = [f"../results/{model}/P01_fft_SR.h5",
               f"../results/{model}/P02_fft_SR.h5",
               f"../results/{model}/P03_fft_SR.h5",
               f"../results/{model}/P04_fft_SR.h5",
               f"../results/{model}/P05_fft_SR.h5",]
    HR = ["../THESIS_INVIVO_DATA/THORAX/P01/h5/P01_cardiac_HR.h5",
               "../THESIS_INVIVO_DATA/THORAX/P02/h5/P02_cardiac_HR.h5",
               "../THESIS_INVIVO_DATA/THORAX/P03/h5/P03_cardiac_HR.h5",
               "../THESIS_INVIVO_DATA/THORAX/P04/h5/P04_cardiac_HR.h5",
               "../THESIS_INVIVO_DATA/THORAX/P05/h5/P05_cardiac_HR.h5",]
    names = ["P01", "P02", "P03", "P04", "P05"]
    peak_frames = [15, 15, 13, 17, 15]
    frames = 25
elif domain == "cerebro":
    predictions = [f"../results/{model}/HV01_fft_SR.h5",
               f"../results/{model}/HV03_fft_SR.h5",
               f"../results/{model}/ICAD28_fft_SR.h5",
               f"../results/{model}/ICAD48_fft_SR.h5",
               f"../results/{model}/ICAD33_fft_SR.h5",]
    HR = ["../THESIS_INVIVO_DATA/BRAIN/Healthy/HV01/h5/HV01_HR.h5",
               "../THESIS_INVIVO_DATA/BRAIN/Healthy/HV03/h5/HV03_HR.h5",
               "../THESIS_INVIVO_DATA/BRAIN/Mild/ICAD28/h5/ICAD28_HR.h5",
               "../THESIS_INVIVO_DATA/BRAIN/Moderate/ICAD48/h5/ICAD48_HR.h5",
               "../THESIS_INVIVO_DATA/BRAIN/Severe/ICAD33/h5/ICAD33_HR.h5",]
    names = ["HV01", "HV03", "ICAD28", "ICAD48", "ICAD33"]
    peak_frames = [1, 5, 0, 1, 1]
    frames = 25

def crop(hr, pred):
    # We assume that if there is a mismatch it's because SR is smaller than HR.
    crop = np.asarray(hr.shape) - np.asarray(pred.shape)
    hr = hr[crop[0]//2:-crop[0]//2,:,:] if crop[0] else hr
    hr = hr[:, crop[1]//2:-crop[1]//2,:] if crop[1] else hr
    hr = hr[:, :, crop[2]//2:-crop[2]//2] if crop[2] else hr
    return hr

def _get_hrsr(ground_truth_file, prediction_file, mask, idx):
    # Use mask to find interesting samples
    sample_pot = np.where(mask == 1)
    # Sample <ratio> samples
    
    

    with h5py.File(prediction_file, mode = 'r' ) as hf:
        sr_u = np.asarray(hf['u'][idx])
        sr_u_all = sr_u[sample_pot[0], sample_pot[1], sample_pot[2]]

        sr_v = np.asarray(hf['v'][idx])
        sr_v_all = sr_v[sample_pot[0], sample_pot[1], sample_pot[2]]

        sr_w = np.asarray(hf['w'][idx])
        sr_w_all = sr_w[sample_pot[0], sample_pot[1], sample_pot[2]]
        
    with h5py.File(ground_truth_file, mode = 'r' ) as hf:
        # Get velocity values in all directions
        hr_u = crop(np.asarray(hf['u'][idx]), sr_u)
        hr_u_all = hr_u[sample_pot[0], sample_pot[1], sample_pot[2]]

        hr_v = crop(np.asarray(hf['v'][idx]), sr_v)
        hr_v_all = hr_v[sample_pot[0], sample_pot[1], sample_pot[2]]
        
        hr_w = crop(np.asarray(hf['w'][idx]), sr_w)
        hr_w_all = hr_w[sample_pot[0], sample_pot[1], sample_pot[2]]
        
        
    return np.concatenate((hr_u_all, hr_v_all, hr_w_all), axis=None), np.concatenate((sr_u_all, sr_v_all, sr_w_all), axis=None)

fig1, sub1 = plt.subplots()
fig2, sub2 = plt.subplots()
k = 0
for prediction, gt, peak in zip(predictions, HR, peak_frames):
    coeff = []
    r = []
    with h5py.File(gt, mode = 'r' ) as hf:
            u = np.asarray(hf['u'])
            frames = u.shape[0]
    for i in range(frames):
        with h5py.File(gt, mode = 'r' ) as hf:
            mask = np.asarray(hf['mask'])
            if len(mask.shape) == 4 and mask.shape[0] != 1:
                mask = mask[i]
            elif len(mask.shape) == 4:
                mask = mask[0]
        hr, sr = _get_hrsr(gt, prediction, mask, i)
        reg = stats.linregress(hr, sr)
        coeff.append(np.round(reg.slope, 3))
        r.append(np.round(reg.rvalue**2, 3))
    print("Subject:", gt)
    print("Peak K:", "{:.3f}".format(np.round(coeff[peak], 3)))
    print("Peak R^2:", "{:.3f}".format(np.round(r[peak], 3)))
    print("Average K:", "{:.3f}".format(np.round(np.mean(coeff), 3)), "+", "{:.3f}".format(np.round(np.std(coeff), 3)))
    print("Average R^2:", "{:.3f}".format(np.round(np.mean(r), 3)), "+", "{:.3f}".format(np.round(np.std(r), 3)))
    sub1.plot(coeff, label=names[k])
    sub2.plot(r, label=names[k])
    k += 1
sub1.set_title("K-value")
sub2.set_title("R^2-value")
sub1.legend()
sub2.legend()
plt.show()
        
    


In [ ]:
a = np.array([1,4,7])
np.mean(a)